In [32]:
import pandas as pd
import sqlite3
import hashlib
import numpy as np
import datetime

In [33]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.options.mode.chained_assignment = None

In [34]:
path = "../../../Databases/football_database.sqlite"
con = sqlite3.connect(path)
cursor = con.cursor()

# Link Satellites

In [35]:
# CREATING LSATS ----------------------------------------------------------#
link_table = "Match"
link_name = "LINK_Match"
link_sat_1 = "LSAT_Match_Statistics" # UNTERTEILUNG DER ATTRIBUTE
link_sat_2 = "LSAT_Match_Bets"

info = pd.read_sql(f"PRAGMA table_info ({link_table})", con = con)
col_names = info["name"]
col_type = info["type"]

In [36]:
info.head()

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,country_id,INTEGER,0,None,0
2,2,league_id,INTEGER,0,None,0
3,3,season,TEXT,0,None,0
4,4,stage,INTEGER,0,None,0


In [48]:
match_attr_cols = col_names[:55].append(col_names[77:85])
match_attr_types = col_type[:85].append(col_type[77:85])

betting_cols = col_names[85:]
betting_types = col_type[:85]


rel_attr_cols = {}
for col_name, col_type in zip(match_attr_cols, match_attr_types):
    if "id" not in col_name:
        rel_attr_cols[col_name] = " " + col_type + "," # add comma

rel_attr_cols

{'season': ' TEXT,',
 'stage': ' INTEGER,',
 'date': ' TEXT,',
 'home_team_goal': ' INTEGER,',
 'away_team_goal': ' INTEGER,',
 'home_player_X1': ' INTEGER,',
 'home_player_X2': ' INTEGER,',
 'home_player_X3': ' INTEGER,',
 'home_player_X4': ' INTEGER,',
 'home_player_X5': ' INTEGER,',
 'home_player_X6': ' INTEGER,',
 'home_player_X7': ' INTEGER,',
 'home_player_X8': ' INTEGER,',
 'home_player_X9': ' INTEGER,',
 'home_player_X10': ' INTEGER,',
 'home_player_X11': ' INTEGER,',
 'away_player_X1': ' INTEGER,',
 'away_player_X2': ' INTEGER,',
 'away_player_X3': ' INTEGER,',
 'away_player_X4': ' INTEGER,',
 'away_player_X5': ' INTEGER,',
 'away_player_X6': ' INTEGER,',
 'away_player_X7': ' INTEGER,',
 'away_player_X8': ' INTEGER,',
 'away_player_X9': ' INTEGER,',
 'away_player_X10': ' INTEGER,',
 'away_player_X11': ' INTEGER,',
 'home_player_Y1': ' INTEGER,',
 'home_player_Y2': ' INTEGER,',
 'home_player_Y3': ' INTEGER,',
 'home_player_Y4': ' INTEGER,',
 'home_player_Y5': ' INTEGER,',
 'hom

In [49]:
# dict to string so I can add it to my create table
var_string_attr = ""
for c, t in rel_attr_cols.items():
    var_string_attr += c + t
var_string_attr

'season TEXT,stage INTEGER,date TEXT,home_team_goal INTEGER,away_team_goal INTEGER,home_player_X1 INTEGER,home_player_X2 INTEGER,home_player_X3 INTEGER,home_player_X4 INTEGER,home_player_X5 INTEGER,home_player_X6 INTEGER,home_player_X7 INTEGER,home_player_X8 INTEGER,home_player_X9 INTEGER,home_player_X10 INTEGER,home_player_X11 INTEGER,away_player_X1 INTEGER,away_player_X2 INTEGER,away_player_X3 INTEGER,away_player_X4 INTEGER,away_player_X5 INTEGER,away_player_X6 INTEGER,away_player_X7 INTEGER,away_player_X8 INTEGER,away_player_X9 INTEGER,away_player_X10 INTEGER,away_player_X11 INTEGER,home_player_Y1 INTEGER,home_player_Y2 INTEGER,home_player_Y3 INTEGER,home_player_Y4 INTEGER,home_player_Y5 INTEGER,home_player_Y6 INTEGER,home_player_Y7 INTEGER,home_player_Y8 INTEGER,home_player_Y9 INTEGER,home_player_Y10 INTEGER,home_player_Y11 INTEGER,away_player_Y1 INTEGER,away_player_Y2 INTEGER,away_player_Y3 INTEGER,away_player_Y4 INTEGER,away_player_Y5 INTEGER,away_player_Y6 INTEGER,away_player_Y7

In [8]:
# create link_table 1
# drop if exists
cursor.execute(f"""DROP TABLE IF EXISTS {link_sat_1};""")

# create the LDTS now
LDTS = datetime.datetime.now()
cursor.execute(f"""CREATE TABLE {link_sat_1}(
    HK VARYINGN CHARACTER(64) NOT NULL PRIMARY KEY,
    HK_{link_name} VARYINGN CHARACTER(64) NOT NULL,
    LDTS DATETIME NOT NULL,
    RS INTEGER NOT NULL,
    HD VARYINGN CHARACTER(64) NOT NULL,
    {var_string_attr}
    FOREIGN KEY(HK_{link_name}) REFERENCES {link_name}(HK)

);""")


rel_bet_cols = {}
for col_name, col_type in zip(betting_cols, betting_types):
    if "id" not in col_name:
        rel_bet_cols[col_name] = " " + col_type + "," # add comma
# dict to string so I can add it to my create table
var_string_bets = ""
for c, t in rel_bet_cols.items():
    var_string_bets += c + t

# drop if exists
cursor.execute(f"""DROP TABLE IF EXISTS {link_sat_2};""")

# create the LDTS now
LDTS = datetime.datetime.now()
cursor.execute(f"""CREATE TABLE {link_sat_2}(
    HK VARYINGN CHARACTER(64) NOT NULL PRIMARY KEY,
    HK_{link_name} VARYINGN CHARACTER(64) NOT NULL,
    LDTS DATETIME NOT NULL,
    RS INTEGER NOT NULL,
    HD VARYINGN CHARACTER(64) NOT NULL,
    {var_string_bets}
    FOREIGN KEY(HK_{link_name}) REFERENCES {link_name}(HK)

);""")

## Filling Link Sats

In [50]:
# FILLING LSATS ------------------------------------------------------------------------------#

# load in corresponding link table
match_df = pd.read_sql(f"SELECT * from {link_table}", con = con) 
link_df = pd.read_sql(f"SELECT * from {link_name}", con = con) 
LDTS = datetime.datetime.now()

# FILL MATCH STATISTICS LSAT
rel_attr_df = match_df[rel_attr_cols.keys()]
rel_attr_df.head() # attr cols

,season,stage,date,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
0,2008/2009,1,2008-08-17 00:00:00,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None
1,2008/2009,1,2008-08-16 00:00:00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None
2,2008/2009,1,2008-08-16 00:00:00,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None
3,2008/2009,1,2008-08-17 00:00:00,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None
4,2008/2009,1,2008-08-16 00:00:00,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None


In [10]:
def hash_row(column_values):
    """function to create a hashkey of a row of a a list of values"""
    string_vars = [str(c).strip().upper() if c is not None else "" for c in column_values]
    string_vars = [c.replace(" ", "_") for c in string_vars]
    # join them
    one_string = "_".join(string_vars)

    # hash gen
    hash_gene = hashlib.new("MD5")

    hash_gene.update(one_string.encode("utf-8")) 

    return hash_gene.hexdigest()

## Fill Link Match Stats Sat

In [ ]:
RS = 1

# hash key from hub
sat_fk = link_df["HK"]

# hash the rows
sat_HD = [hash_row(rel_attr_df.iloc[i]) for i in range(len(rel_attr_df))]

# compute the Sat_HK
sat_HK = [hash_row([sat_fk.iloc[i], LDTS, RS]) for i in range(len(sat_fk))]

# create the sat df
rel_attr_df["HK"] = sat_HK
rel_attr_df["HK_" + link_name] = sat_fk
rel_attr_df["LDTS"] = LDTS
rel_attr_df["RS"] = RS
rel_attr_df["HD"] = sat_HD

rel_attr_df.head()

In [ ]:
# into the database
rel_attr_df.to_sql(name = link_sat_1,con=con, if_exists='append', index = False)

## Fill Link Bet Satellite

In [ ]:
# FILL MATCH BET LSAT
rel_attr_df = match_df[rel_bet_cols.keys()]

# hash key from hub
sat_fk = link_df["HK"]

# hash the rows
sat_HD = [hash_row(rel_attr_df.iloc[i]) for i in range(len(rel_attr_df))]

# compute the Sat_HK
sat_HK = [hash_row([sat_fk.iloc[i], LDTS, RS]) for i in range(len(sat_fk))]

# create the sat df
rel_attr_df["HK"] = sat_HK
rel_attr_df["HK_" + link_name] = sat_fk
rel_attr_df["LDTS"] = LDTS
rel_attr_df["RS"] = RS
rel_attr_df["HD"] = sat_HD

# into the database
rel_attr_df.to_sql(name = link_sat_2,con=con, if_exists='append', index = False)

# Business Satellites -> XML Extracting

In [3]:
# IMPORT DATA ------------------------------------------------------------------#
# import the business satelites dfs

card_df = pd.read_csv("../../Business_Satelite_Data/card_df.csv", index_col = 0, low_memory=False)
corner_df = pd.read_csv("../../Business_Satelite_Data/corner_df.csv", index_col = 0, low_memory=False)
cross_df = pd.read_csv("../../Business_Satelite_Data/cross_df.csv", index_col = 0, low_memory=False)
foulcommit_df = pd.read_csv("../../Business_Satelite_Data/foulcommit_df.csv", index_col = 0, low_memory=False)
goal_df = pd.read_csv("../../Business_Satelite_Data/goal_df.csv", index_col = 0, low_memory=False)
possession_df = pd.read_csv("../../Business_Satelite_Data/possession_df.csv", index_col = 0, low_memory=False)
shotoff_df = pd.read_csv("../../Business_Satelite_Data/shotoff_df.csv", index_col = 0, low_memory=False)
shoton_df = pd.read_csv("../../Business_Satelite_Data/shoton_df.csv", index_col = 0, low_memory=False)

# example
card_df.head()

,card_number,match_api_id,card_type,comment,elapsed,event_incident_typefk,card_id,n,player1,sortorder,stats,subtype,team,type,elapsed_plus,del,goal_type
0,1,489042,y,y,78,73,379481,342,24157,1,NaN,serious_fouls,10260.0,card,NaN,NaN,NaN
1,2,489042,y,y,82,73,379503,346,30362,1,NaN,serious_fouls,10260.0,card,NaN,NaN,NaN
2,3,489042,y,y,90,70,379547,353,30829,1,NaN,NaN,10260.0,card,NaN,NaN,NaN
3,1,489044,y,y,56,73,377978,327,37442,5,NaN,serious_fouls,8650.0,card,NaN,NaN,NaN
4,2,489044,y,y,90,25,378060,353,46621,3,NaN,stall_time,8650.0,card,NaN,NaN,NaN


In [85]:
match_df = pd.read_sql("select * from Match", con = con)
link_match_df = pd.read_sql("select * from LINK_Match", con = con)

match_ids = match_df["match_api_id"].values
match_hks = link_match_df["HK"].values

match_id_hk = pd.DataFrame({"match_api_id" : match_ids, 
                   "HK_Link_Match" : match_hks})

# MERGING ------------------------------------------------------------------------------------#

df_dict = {}
# loop over the business satelite data
business_sat_data = {"card_df" : card_df,
                     "cross_df" : cross_df,
                     "corner_df" : corner_df,
                     "foulcommit_df" : foulcommit_df,
                     "goal_df" : goal_df,
                     "possession_df" : possession_df,
                     "shotoff_df" : shotoff_df,
                     "shoton_df" : shoton_df}

for name, df in business_sat_data.items():
    merged_df = df.merge(match_id_hk, how = "inner" , right_on= "match_api_id",left_on="match_api_id")
    df_dict[name] = merged_df
    
df_dict[name].head() # foregn key link_match is added


,shoton_number,match_api_id,elapsed,event_incident_typefk,shoton_id,n,player1,sortorder,stats,subtype,team,type,elapsed_plus,goal_type,del,coordinates,card_type,HK_Link_Match
0,1,489042,3,61,378828,253,24154.0,0,NaN,blocked_shot,10260.0,shoton,NaN,NaN,NaN,NaN,NaN,17cbde77fb945a71f976c232b4d48210
1,2,489042,7,154,378866,258,24157.0,2,NaN,header,10260.0,shoton,NaN,NaN,NaN,NaN,NaN,17cbde77fb945a71f976c232b4d48210
2,3,489042,14,153,378922,274,30829.0,1,NaN,shot,10260.0,shoton,NaN,NaN,NaN,NaN,NaN,17cbde77fb945a71f976c232b4d48210
3,4,489042,14,153,378923,279,30373.0,2,NaN,shot,10260.0,shoton,NaN,NaN,NaN,NaN,NaN,17cbde77fb945a71f976c232b4d48210
4,5,489042,17,137,378951,272,30373.0,3,NaN,distance,10260.0,shoton,NaN,NaN,NaN,NaN,NaN,17cbde77fb945a71f976c232b4d48210


In [51]:
#hash keys -----------------------------------------------------
for name, df in df_dict.items():
    hk_df = df[[name.split("_")[0] + "_id", "HK_Link_Match"]]
    hk_df["RS"] = 1
    sat_hk = [hash_row(hk_df.iloc[i]) for i in range(len(hk_df))]
    sat_attr_df = df.drop(columns = ["HK_Link_Match", name.split("_")[0] + "_id"])
    sat_HD = [hash_row(sat_attr_df.iloc[i]) for i in range(len(sat_attr_df))]

    # add new cols
    df.insert(0, "HK", sat_hk)
    df.insert(1, "LDTS", datetime.datetime.now())
    df.insert(2, "RS", RS)
    df.insert(3, "HD", sat_HD)
    	
    # to sql
    df.to_sql(name = "B_LSAT_" + name.split("_")[0] ,con=con, if_exists='replace', index = False,
                  dtype={"HK": "VARYINGN CHARACTER(64) NOT NULL PRIMARY KEY",
                         "HK_Link_Match" : "VARYINGN CHARACTER(64) NOT NULL",
                         "LDTS": "DATETIME NOT NULL",
                         "HD" : "VARYINGN CHARACTER(64) NOT NULL",
                         "RS": "INT NOT NULL"
                         })

NameError: name 'df_dict' is not defined

In [ ]:
df.head()